In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/titanik.csv", delimiter=';')
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Скорее всего удалим Cabin, дозаполним Age, удалим пару объектов из выборки для Embarked

### Вспомогательные преобразования

Заменим категориальные признаки в столбцах

In [4]:
df['Sex'] = df['Sex'].apply(lambda x: 1 if x == 'male' else 0)

Удалим ненужные столбцы Name, PassengerId, Cabin, Ticket

In [5]:
df.drop(['PassengerId', 'Name','Cabin','Ticket'], axis=1, inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(5), object(1)
memory usage: 55.8+ KB


В столбце Embarked просто выкинем строки с пропусками:

In [7]:
df = df[df['Embarked'].notna()]

В столбце 'Embarked' три уникальных значения, которые являются

In [8]:
print(*df['Embarked'].unique())

S C Q


In [9]:
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,False,True
1,1,1,0,38.0,1,0,71.2833,False,False
2,1,3,0,26.0,0,0,7.9250,False,True
3,1,1,0,35.0,1,0,53.1000,False,True
4,0,3,1,35.0,0,0,8.0500,False,True
...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,False,True
887,1,1,0,19.0,0,0,30.0000,False,True
888,0,3,0,NaN,1,2,23.4500,False,True
889,1,1,1,26.0,0,0,30.0000,False,False


Далее, после OHE необходимо преобразовать данные с помощью MinMaxScaler, чтобы дата находилась на отрезке [0,1]

In [10]:
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,0.0,1.0,1.0,0.271174,0.125,0.0,0.014151,0.0,1.0
1,1.0,0.0,0.0,0.472229,0.125,0.0,0.139136,0.0,0.0
2,1.0,1.0,0.0,0.321438,0.000,0.0,0.015469,0.0,1.0
3,1.0,0.0,0.0,0.434531,0.125,0.0,0.103644,0.0,1.0
4,0.0,1.0,1.0,0.434531,0.000,0.0,0.015713,0.0,1.0


### Заполнение пропусков Age

In [11]:
updated_df = df.copy()
updated_df['Age'] = updated_df['Age'].fillna(updated_df['Age'].mean())

Далее стандартные манипуляции train и test

In [12]:
X, y = updated_df.drop('Survived', axis=1), updated_df['Survived']

cv = KFold(n_splits=3, shuffle=True, random_state=42)
cv_score = cross_val_score(LogisticRegression(), cv=cv, X=X, y=y)
cv_score.mean()

np.float64(0.7997732581065914)

#### Воспользуемся предсказанием пропущенных значений с помощью линейной модели

In [13]:
df_numeric = df.copy()

train_df = df_numeric[df_numeric['Age'].isnull() == False]
test_df = df_numeric[df_numeric['Age'].isnull() == True]

y = train_df['Age']

train_df.drop(['Age', 'Survived'], axis=1, inplace=True)
test_df.drop(['Age', 'Survived'], axis=1, inplace=True)

C:\Users\kaban\AppData\Local\Temp\ipykernel_2268\2014924164.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(['Age', 'Survived'], axis=1, inplace=True)
C:\Users\kaban\AppData\Local\Temp\ipykernel_2268\2014924164.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(['Age', 'Survived'], axis=1, inplace=True)


Обучаем теперь модель линейной регрессии

In [14]:
model = LinearRegression()

model.fit(train_df, y)

prediction = model.predict(test_df)

In [15]:
updated_df2 = df.copy()

indices = test_df.index

updated_df2.loc[indices, 'Age'] = prediction

In [16]:
updated_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    889 non-null    float64
 1   Pclass      889 non-null    float64
 2   Sex         889 non-null    float64
 3   Age         889 non-null    float64
 4   SibSp       889 non-null    float64
 5   Parch       889 non-null    float64
 6   Fare        889 non-null    float64
 7   Embarked_Q  889 non-null    float64
 8   Embarked_S  889 non-null    float64
dtypes: float64(9)
memory usage: 62.6 KB


### Определим теперь качество модели на датасете

In [17]:
X = updated_df2.drop('Survived', axis=1)
y = updated_df2['Survived']

cv_score2 = cross_val_score(LogisticRegression(), X, y, cv=cv)
cv_score2.mean()

np.float64(0.7975210058543393)

Вывод: машинное обучение в данном случае не требуется для заполнения пропусков. Можно остановиться на среднем